# ANALYSIS OF REAL DATA (GSFC)

From real MnKa1, MnKa2 (+other lines) data, it creates a library of optimal filters and reconstruct (and calibrate) data calculating FWHM of lines

**Energy units are (k)eV**

Imports and definitions

PREPROCESSING

1. Clean multiple-pulse records

    1.1. Detect pulses in records file
     
    1.2. Identify multiple pulse records
    
    1.3. Remove multiple-pulse records (to select only single records)

2. Monocrhomatic (Mn Ka1) library creation

    2.1. Plot histogram of maximum values in single-pulse records
    
    2.2. Select records around Ka1 and Ka2 lines (by limiting Pulse Height)
    
    2.3. Create a library with this new file with Kas photons in single records
    
    2.4. Reconstruct Kas data with initial Kas library
    
    2.5. Read data from HR Kas evt file to identify Ka1 and Ka2
    
    2.6. Fit a double Gaussian to select ka1+Ka2 pulses
    
    2.7. Select PH_ID of (non)Ka1 pulses to later exclude them
    
    2.8. Extract Ka1 pulses to build a new more monochromatic library
    
    2.9. Build the new libary of Ka1 pulses 
        

## Imports and definitions

In [1]:
from subprocess import check_call, STDOUT
import os
from astropy.io import fits
import numpy.polynomial.polynomial as poly
from numpy.polynomial import Polynomial as P
import tempfile
from datetime import datetime
import shutil, shlex
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas
from fit2GaussAndRatio import fit2GaussAndRatio
from getMaximaDensity import getMaximaDensity
from fit2gauss2hist import fit2gauss2hist
from fit3gauss2hist import fit3gauss2hist
from gainScaleFit import gainScaleLinearFit, gainScalePolyFit
from fitVoigt2hist import fitVoigt2hist
from jitterCorr import jitterCorr
from baseCorr import baseCorr
from GSFC import averagePulse, autoDeterminePulseWindowAndThreshold, categorize, deviatonFromAveragePulse,nrecs,chi2
from commands import run_comm
from clean_records import remove_invalid_records
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
from numpy import random
from calibLines import *
from annote import AnnoteFinder
from scipy.interpolate import interp1d


import ipywidgets as widgets
%matplotlib widget

cwd = os.getcwd()
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
xmlfileSX = os.environ["SIXTE"] + "/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml"
print(xmlfileSX)

/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml


In [2]:
# data files for library creation
channels = (1,3,5,7,9,11,13,15,17,19)
channel = 1
resDir = "channel_" + str(channel)
if not os.path.exists(resDir):
    os.makedirs(resDir)
ratio = 5 # ratio Ka1_gaussProb/Ka2_gaussProb
fileph  = "pulse/pulse_chan" + str(channel) + ".fits" # initial data file with all records and PH_ID column populated
fileph_singles = resDir + "/pulse_chan" + str(channel) + "_singles.fits" # initial data file with all records and PH_ID column populated
filephHR  = resDir + "/pulse_chan" + str(channel) + "_HR.fits" # initial data file with all records and PH_ID column populated and High Res events
fileph_KasPKH = resDir + "/pulse_chan" + str(channel) + "_KasPKH.fits" # data file with only those records with Kas lines selected by Peak Heigth
fileph_Kas = resDir + "/pulse_chan" + str(channel) + "_Kas.fits" # data file with only those records with Kas lines 
noiseph = "noise/noise_chan" + str(channel) + ".fits"
noisefile = "noise/noise_chan" + str(channel) + "_spec.fits"

In [63]:
# select SIRENA parameters for library Kas creation and reconstruction of data files
samprate=195312.5
plen = 4096
oflen = 4096
liblen = 4096 # length of maximum optimal filter for library creation
preBuffer = 0
pBstr = ""
if preBuffer > 0:
    pBstr = "_pB" + str(preBuffer)
method = "OPTFILT"
F0orB0 = "F0"
nS = 5
sU = 3
sD = 4
#KaseV = 5895 # eV reference energy: 8.2%*5.88765(Ka2)+16.2%*5.89875(Ka1) for initial Kas library
libKasPKH = resDir + "/" + "library_KasPKH_" + str(liblen) + ".fits" # initial lib with PKH selected Kas pulses
libKas = resDir + "/" + "library_Kas_" + str(liblen) + ".fits" # final lib of Kas pulses (PH+clean) selected pulses

In [4]:
# reconstructed files
evtKasPKH_libKas = resDir + "/" + "evtKasPKH_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + ".fits"
evtKasPKH_libKas_HR = resDir + "/" + "evtKasPKH_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + "_HR.fits"
evtKas_libKas = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + ".fits"
evtKas_libKas_HR = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + "_HR.fits"

In [5]:
# library Ka1 creation
fileph_Ka1 = resDir + "/" + "pulse_chan" + str(channel) + "_Ka1.fits" # data file with only those single records with Ka1 lines
fileph_Ka1_clean = resDir + "/" + "pulse_chan" + str(channel) + "_Ka1_clean.fits" # data file with only those single-valid records with Ka1 lines
libKa1 = resDir + "/" + "library_Ka1_"+ str(liblen) + "_chan" + str(channel) +  pBstr + ".fits"
libKa1_cleanp = resDir + "/" + "library_Ka1_"+ str(liblen) + "_chan" + str(channel) +  pBstr + "_cleanp.fits"

## PREPROCESSING

## 1. Clean multiple-pulse records

### 1.1) Detect pulses in records file

In [ ]:
#%%script false --no-raise-error
# build fake library to detect pulses and identify multiple-pulse records
tmpFile = resDir + "/" + "detections0.fits"
tmpFile2 = resDir + "/" + "fakeLib.fits"
comm = ("tesreconstruction Recordfile=" + fileph + " TesEventFile=" + tmpFile + " PulseLength=" + str(liblen) + 
        " LibraryFile=" + tmpFile2 + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + 
        " opmode=0 FilterMethod=" + F0orB0 + " clobber=yes EnergyMethod=" + method + " NoiseFile=" + noisefile +
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(MnKas_cmass) + " preBuffer=" + str(preBuffer) + " OFLength=" + str(oflen))
try:
    print("Do initial detection")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Building initial detection-library with command:\n", comm)
    raise
os.remove(tmpFile2)
print("##########################################")
print("Finished creation of fake detection-Library")
print("##########################################")

### 1.2) Identify multiple pulse records

In [ ]:
#%%script false --no-raise-error
pulsesFile = resDir + "/pulses.txt"
# dump records info
colname = "'SIGNAL, PH_ID, GRADE1, GRADE2'" 
comm = ("fdump wrap=yes infile=" + tmpFile + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=" + pulsesFile + " clobber=yes")
try:
    print("FDUMPing evt file")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error FDUMPing evt file with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise
    


In [ ]:
#%%script false --no-raise-error

# find single records
dataAll = pandas.read_csv(pulsesFile, skiprows=0, sep="\s+")
display(dataAll)
n_ocurr = dataAll.PH_ID.value_counts() # number of ocurrences of each PH_ID
#print(n_ocurr)
all_PH_ID = np.unique(dataAll.PH_ID.to_list())   # all PH_ID 
single_PH_ID = list()             # single records 
multiple_PH_ID = list()             # multiple records 
for key in sorted(n_ocurr.keys()):
    if n_ocurr[key] == 1:
        #print("Single record for PH_ID=", key)
        single_PH_ID.append(key)
    else:
        multiple_PH_ID.append(key)
os.remove(pulsesFile)

print("Number of records in",fileph, "=", len(all_PH_ID))
print("Number of Single records in",fileph, "=", len(single_PH_ID))
#print(single_PH_ID)
print("Number of Multiple records in",fileph, "=", len(multiple_PH_ID))
#print(multiple_PH_ID)

### 1.3) Remove multiple pulse records (to select only single records)

In [ ]:
%%script false --no-raise-error

remove_invalid_records(infile=fileph, ext=1, id_list=multiple_PH_ID, colname="PH_ID", outfile=fileph_singles)

## 2. Monochromatic (Ka1) library creation

### 2.1) Plot histogram of maximum values in single-pulse records

In [6]:
plt.close()
f = fits.open(fileph_singles)
ADCdata = f["TESRECORDS"].data['ADC']
baselines = np.mean(ADCdata[:,0:1950], axis=1)
ADCmax = np.amax(ADCdata, axis=1)

fig = plt.figure(figsize=(9,4))
ax1 = fig.add_subplot(1, 2, 1)
bin_heights, bin_borders, _ = ax1.hist(ADCmax, bins=20, alpha=0.4)
ax1.set_xlabel("Maximum value of ADC in record")
ax1.set_ylabel("Number of records")
ax1.set_title("Histogram of max(ADC)")
PHmin = 22500 # ADC units to limit Kas lines
ax1.axvline(PHmin, linestyle="--", color="gray")
PHmax = 25000 # ADC units to limit Kas lines
ax1.axvline(PHmax, linestyle="--", color="gray")

# Identify 'unusual' baselines (fragments of pulses)
ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(baselines, bins=25, alpha=0.4)
ax2.set_title("Histogram of record baselines")
ax2.set_xlabel("Mean value of baseline")
ax2.annotate('partial initial (undetected) pulse', xy=(9500,10000), xytext=(9000, 30000),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )
print("Max baseline=", np.max(baselines))
print("Indices of baselines:", np.where(baselines>8400))
print("Largest baselines:", baselines[baselines>8400])
f.close()
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Max baseline= 10160.496
Indices of baselines: (array([ 1176,  1178,  1179, 13337, 22549, 26148, 30103, 41365]),)
Largest baselines: [ 8400.724  8400.491  8400.977  8752.445  8478.793 10160.496  9010.425
  8465.033]


### 2.2) Select records around Ka1 and Ka2 lines (by limiting max(ADC))

In [7]:
#%%script false --no-raise-error

comm = ("fselect  infile=" + fileph_singles + "+1 outfile=" + fileph_KasPKH + " expr='max(ADC)>" + str(PHmin) + " && max(ADC)<" + str(PHmax) + "' clobber=yes")
try:
    print("Selecting Kas by Pulse Height")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Selecting Kas by Pulse Height with command:\n", comm)
    shutil.rmtree(tmpDir)
    raise

Selecting Kas by Pulse Height
fselect  infile=channel_1/pulse_chan1_singles.fits+1 outfile=channel_1/pulse_chan1_KasPKH.fits expr='max(ADC)>22500 && max(ADC)<25000' clobber=yes


### 2.3) Create a library with this new file with Kas photons in single records

In [ ]:
%%script false --no-raise-error
# calculate noise spectrum
comm = ("gennoisespec inFile=" + noiseph + " outFile=" + noisefile + " intervalMinSamples=" + str(liblen) +
        " nintervals=1830 pulse_length= clobber=yes samplingRate=" + samprate)
try:
    print("Getting noise spectrum")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error building noise spectrum:\n", comm)
    shutil.rmtree(tmpDir)
    raise

In [8]:
#%%script false --no-raise-error

# build library Kas
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
comm = ("tesreconstruction Recordfile=" + fileph_KasPKH + " TesEventFile=" + tmpFile + " PulseLength=" + str(plen) + 
        " LibraryFile=" + libKasPKH + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + 
        " opmode=0 FilterMethod=" + F0orB0 + " clobber=yes EnergyMethod=" + method + 
        " OFLength=" + str(liblen) + " NoiseFile=" + noisefile +
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(MnKas_cmass) + " preBuffer=" + str(preBuffer))
run_comm(comm, "Building initial library (Kas)")
os.remove(tmpFile)
print("##########################################")
print("Finished creation of Library of Kas events")
print("##########################################")

Building initial library (Kas)
tesreconstruction Recordfile=channel_1/pulse_chan1_KasPKH.fits TesEventFile=channel_1/pp1593599433.fits PulseLength=4096 LibraryFile=channel_1/library_Kas_4096.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=0 FilterMethod=F0 clobber=yes EnergyMethod=OPTFILT OFLength=4096 NoiseFile=noise/noise_chan1_spec.fits XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml monoenergy=5894.40 preBuffer=0
##########################################
Finished creation of Library of Kas events
##########################################


### 2.4) Reconstruct Kas data with initial (Kas) library

In [9]:
#%%script false --no-raise-error
comm = ("tesreconstruction Recordfile=" + fileph_KasPKH + " TesEventFile=" + evtKasPKH_libKas + " PulseLength=" + str(plen) +
        " LibraryFile=" + libKasPKH + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + " opmode=1 " +
        " clobber=yes EnergyMethod=" + method + " filtEeV=" + str(MnKas_cmass) + " OFStrategy=FREE OFLib=yes OFLength=" + str(liblen) + 
        " preBuffer=" + str(preBuffer) + " XMLFile=" + xmlfileSX)
run_comm(comm, "Reconstructing Kas data w/ initial library (Kas)")

# Select only HR events
comm = ("fselect  infile=" + evtKasPKH_libKas + " outfile=" + evtKasPKH_libKas_HR + " expr='GRADE1 == " + str(liblen) + 
       " && GRADE2 > 500' clobber=yes")
run_comm(comm, "Selecting HR Kas evts")

Reconstructing Kas data w/ initial library (Kas)
tesreconstruction Recordfile=channel_1/pulse_chan1_KasPKH.fits TesEventFile=channel_1/evtKasPKH_pulse_chan1_libKas_pL4096_OPTFILT4096.fits PulseLength=4096 LibraryFile=channel_1/library_Kas_4096.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=1  clobber=yes EnergyMethod=OPTFILT filtEeV=5894.40 OFStrategy=FREE OFLib=yes OFLength=4096 preBuffer=0 XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml
Selecting HR Kas evts
fselect  infile=channel_1/evtKasPKH_pulse_chan1_libKas_pL4096_OPTFILT4096.fits outfile=channel_1/evtKasPKH_pulse_chan1_libKas_pL4096_OPTFILT4096_HR.fits expr='GRADE1 == 4096 && GRADE2 > 500' clobber=yes


### 2.5) Read data from HR Kas evt file to identify Ka1 and Ka2

In [10]:
#%%script false --no-raise-error
pulseFile = resDir + "/pulsesKas.txt"
colname = "'SIGNAL, PH_ID, GRADE1, GRADE2, PHI'" 
comm = ("fdump wrap=yes infile=" + evtKasPKH_libKas_HR + "+1 columns=" + colname + " pagewidth=256 rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=" + pulseFile + " clobber=yes")
run_comm(comm, "FDUMPing evt file")

dataKasPKH_HR = pandas.read_csv(pulseFile, skiprows=0,sep="\s+")
os.remove(pulseFile)
print("\nNumber of PKH-HR Kas pulses in",fileph_KasPKH, "=", len(dataKasPKH_HR))
display(dataKasPKH_HR)
print("Kas events with PHI==0. (double pulses):", dataKasPKH_HR[dataKasPKH_HR.PHI == 0.])

FDUMPing evt file
fdump wrap=yes infile=channel_1/evtKasPKH_pulse_chan1_libKas_pL4096_OPTFILT4096_HR.fits+1 columns='SIGNAL, PH_ID, GRADE1, GRADE2, PHI' pagewidth=256 rows='-' prhead=no showcol=yes showunit=no showrow=no outfile=channel_1/pulsesKas.txt clobber=yes

Number of PKH-HR Kas pulses in channel_1/pulse_chan1_KasPKH.fits = 18707


,SIGNAL,PH_ID,GRADE1,GRADE2,PHI
0,5.851242,0,4096,4096,-0.430597
1,5.839119,1,4096,4096,-0.082949
2,5.848833,2,4096,4096,0.225297
3,5.846148,3,4096,4096,-0.170477
4,5.846350,4,4096,4096,0.501811
...,...,...,...,...,...
18702,6.333599,47014,4096,4096,0.184659
18703,5.844242,47015,4096,4096,0.461042
18704,5.853975,47016,4096,4096,-0.065375
18705,5.848557,47017,4096,4096,-0.204660


Kas events with PHI==0. (double pulses): Empty DataFrame
Columns: [SIGNAL, PH_ID, GRADE1, GRADE2, PHI]
Index: []


### 2.6) Select Ka1+Ka2 pulses according to PH

In [11]:
#%%script false --no-raise-error
plt.close()
# select data in Kas range (otherwise there is always a smaller pulse comimg from multiple-pulse rows)
# print(min(dataKas_HR.SIGNAL), max(dataKas_HR.SIGNAL))
PHminKas, PHmaxKas = 5.82, 5.875
data_Ka2Ka1 = dataKasPKH_HR[(dataKasPKH_HR.SIGNAL>PHminKas) & (dataKasPKH_HR.SIGNAL<PHmaxKas)]
print("Number of PH-Kas pulses:", len(data_Ka2Ka1))
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot(1, 1, 1)
ax.hist(data_Ka2Ka1.SIGNAL, bins=50, alpha=0.4, density=True)
ax.set_xlabel("Reconstructed PH (a.u.)")
ax.set_ylabel("# photons")
ax.set_title("Mn Ka photons")

Number of PH-Kas pulses: 10630


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Mn Ka photons')

### 2.7) Create data file with pulses only from Mn Kas

#### 2.7.1) Select Kas events according to reconstructed PH

In [12]:
#According to ENERGY value: select PH_IDs to exclude non-truly-Kas in single-pulse records
PH_ID_Kas = np.sort(data_Ka2Ka1.PH_ID.to_list())
PH_ID_noKas = np.sort(list(set(dataKasPKH_HR.PH_ID.to_list()).difference(PH_ID_Kas)))

print("Number of PKH-Kas pulses:", len(dataKasPKH_HR.SIGNAL))
print("Number of PH-Kas pulses:", len(data_Ka2Ka1.SIGNAL))
print("Number of PKH-Kas pulses which are non-truly PH-Kas pulses:", len(PH_ID_noKas))
print("Example of PH-Kas pulses (PH_ID):", PH_ID_Kas[:20])
print("Example of PKH-Kas pulses which are non-truly Kas pulses (PH_ID):", PH_ID_noKas[:10])

Number of PKH-Kas pulses: 18707
Number of PH-Kas pulses: 10630
Number of PKH-Kas pulses which are non-truly PH-Kas pulses: 8077
Example of PH-Kas pulses (PH_ID): [ 0  1  2  3  4  5  6  8  9 11 12 15 16 17 18 20 21 23 24 26]
Example of PKH-Kas pulses which are non-truly Kas pulses (PH_ID): [10 14 22 25 44 46 54 55 59 61]


In [14]:
# select data based on PH
f = fits.open(fileph_KasPKH)
ADCdata_KasPKH = f["TESRECORDS"].data['ADC']
PHIDdata_KasPKH = f["TESRECORDS"].data['PH_ID']

PHIDdata_KasPH = PHIDdata_KasPKH[np.isin(PHIDdata_KasPKH,PH_ID_Kas)]
ADCdata_KasPH = ADCdata_KasPKH[np.isin(PHIDdata_KasPKH,PH_ID_Kas)]
f.close()
print("PHID of PKH-Kas pulses:", PHIDdata_KasPKH[0:20])
print("PHID of PH-Kas pulses:", PHIDdata_KasPH[0:20])

PHID of PKH-Kas pulses: [ 0  1  2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 20 21 22]
PHID of PH-Kas pulses: [ 0  1  2  3  4  5  6  8  9 11 12 15 16 17 18 20 21 23 24 26]


#### 2.7.2) template of PH-Kas pulses: pulses will be compared with template

In [40]:
nrecords = ADCdata_KasPH.shape[0]
lenrec = ADCdata_KasPH.shape[1]
len_ave = 5000
ave_KasPH = np.zeros(len_ave)
fig = plt.figure(figsize=(6,3))
ax1 = fig.add_subplot(1, 1, 1)

istart, iend, thresh, ave_KasPH = autoDeterminePulseWindowAndThreshold(ADCdata_KasPH, 
                                nbase=700, nsigma=100, numSamples=len_ave, plot=True, ax=ax1)
# autodeterminePulseAndWindow places start of template in sample=1000 

# get exact point where template crosses threshold to better align pulses later
icross = 999 + (thresh-ave_KasPH[999])/(ave_KasPH[1000]-ave_KasPH[999])
nobsln_KasPH = np.zeros((ADCdata_KasPH.shape)) # remove baseline
ax1.plot(icross,thresh,marker='x')

print("Pulse window in average:", istart, iend, thresh)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Possible pulse in preBuffer:
      For ip= 6014 s0,ini,fin= 0 -1000 4000
      Pulse not considered in average
Pulse window in average: 1001 1532 1963.8806651908226


#### 2.7.2) Clean Kas events according GSFC criteria

In [41]:
invalid_records = list()

GSFCrejected = list()
GSFCrejected_comm = list()
for irec in range(nrecords):
    base = np.mean(ADCdata_KasPH[irec,1000:1800])
    nobsln_KasPH[irec,:] = ADCdata_KasPH[irec,:] - base 
    #print("For irec=",irec,"base=",np.mean(nobsln_Ka1[irec,1000:1800]))
    rec_class = categorize(nobsln_KasPH[irec,:], istart, iend, thresh, joff=10) # initial offset is needed to avoid jitter effects
    #print("irec=", irec, "rec_class=", rec_class)
    if rec_class["rejected"] == 1:
        GSFCrejected.append(irec)
        GSFCrejected_comm.append(rec_class["rejected_comm"])
print("List of rejected pulses:", GSFCrejected)                        
print("Cause of rejection:", GSFCrejected_comm)            

invalid_records = GSFCrejected

List of rejected pulses: [340, 784, 1425, 1943, 2296, 2511, 2670, 5166, 5965, 6234, 6442, 7023, 7181, 7243, 7958, 8166, 9723]
Cause of rejection: ['Above thres in i=1544 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]', 'Above thres in i=1540 and rec[i]>rec[i-1]', 'Above thres in i=1536 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]', 'Above thres in i=1537 and rec[i]>rec[i-1]', 'Above thres in i=1537 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]', 'Above thres in i=1536 and rec[i]>rec[i-1]', 'Above thres in i=1539 and rec[i]>rec[i-1]', 'Above thres in i=1541 and rec[i]>rec[i-1]', 'Above thres in i=1536 and rec[i]>rec[i-1]', 'Above thres in i=1540 and rec[i]>rec[i-1]', 'Above thres in i=1537 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]', 'Above thres in i=1538 and rec[i]>rec[i-1]']


#### 2.7.3) Clean Kas events according to $\chi^2$ value of comparison with template

In [57]:
# align pulses with average record and calculate chisq
chisq = np.zeros(nrecords)
nobsln_KasPH_cut = np.zeros((nrecords, len_ave))
meanstd = 8.7
chth = 1e4 # threshld in Chisq to remove records
chi2rejected = list()
icross_cut = list()
for irec in range(nrecords):
    s0 = np.where(nobsln_KasPH[irec,:] > thresh)[0][0]
    stSample = s0-istart
    fnSample = stSample + len_ave
    if stSample < 0:
        print("Possible pulse in preBuffer:")
        print("      For irec=",irec,"s0,ini,fin=", s0,stSample,fnSample)
        chi2rejected.append(irec)
        chisq[irec] = 1e6
        icross_cut.append(1000)
    else:
        nobsln_KasPH_cut[irec,:] = nobsln_KasPH[irec,stSample:fnSample]
        # assuming template window IS IN SAMPLE=1000
        ic = 999 + (thresh-nobsln_KasPH_cut[irec,999])/(nobsln_KasPH_cut[irec,1000]-nobsln_KasPH_cut[irec,999])
        icross_cut.append(ic)
        off = icross-ic
        xx = np.array(range(len_ave))
        off_func = interp1d(xx, nobsln_KasPH_cut[irec,:], kind='linear', fill_value='extrapolate')
        off_cut = off_func(xx-off)
        chisq[irec] = chi2(off_cut, ave_KasPH, meanstd)
        if chisq[irec] > chth:
            chi2rejected.append(irec)

#### 2.7.4) Plot results of pulse qualification

In [61]:
# Plots of selection of pulses
plt.close()
fig = plt.figure(figsize=(9,6))

# 1) Plot difference between average pulse and rejected/invalid/tocheck pulses
# -----------------------------------------------------------------------------
test_pulses = [3317]
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(range(len(ave_KasPH)), len(ave_KasPH)*[0], ls='')
for irej in test_pulses:
    index = test_pulses.index(irej)
    color = "C"+ str(index)
    xx = np.array(range(len_ave))
    off_func = interp1d(xx, nobsln_KasPH_cut[irej,:], kind='linear', fill_value='extrapolate')
    off = icross-icross_cut[irej]
    off_rej = off_func(xx-off)
    diff_record = off_rej - ave_KasPH
    ax1.plot(range(len_ave), diff_record, color=color, label=("Test pulse (" + str(irej) + ")"))
ax1.axvline(istart, color="gray", ls="--")
ax1.axvline(iend, color="gray", ls="--")
ax1.set_title("Difference with Average pulse")
ax1.set_ylabel("Record in ADC - Average (a.u.)")
ax1.set_xlabel("Record sample")
ax1.legend(fontsize="xx-small", loc="right")

# 2) Average and test pulse
# ----------------------------
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(range(len(ave_KasPH)), ave_KasPH, label=("Average pulse"), color="black")
for irej in test_pulses:
    index = test_pulses.index(irej)
    color = "C"+ str(index)
    rej_record = nobsln_KasPH_cut[irej,:]
    ax2.plot(range(len(ave_KasPH)), rej_record, label=("Test pulse (" + str(irej) + ")"), color=color)
    ax2.plot(icross_cut[irej], thresh, color=color, marker="x")
    off = icross-icross_cut[irej]
    ax2.plot(np.arange(len(ave_KasPH))+off, rej_record, label=("Test pulse (" + str(irej) + ") offset"), color=color, ls="--")
ax2.axvline(istart, color="gray", ls="--")
ax2.axvline(iend, color="gray", ls="--")
ax2.axhline(thresh, color="gray", ls="--")
ax2.set_title("Average & Test pulse")
ax2.set_ylabel("ADC (a.u.)")
ax2.set_xlabel("Record sample")
ax2.legend(fontsize="xx-small")

# 3) Cumulative Chisq deviation between average record and every record
# ----------------------------------------------------------------------
ax3 = fig.add_subplot(2, 2, 3)
nofrecs = nrecs(chisq, chisq)
ax3.plot(nofrecs, chisq, marker='.', ls='')
ax3.fill_between(nofrecs, min(chisq), max(chisq), where=chisq > chth, color='gray', alpha=0.05)
ax3.set_yscale('log')
ax3.set_ylabel("Deviation")
ax3.set_xlabel("# records to remove (Dev>deviation)")

# 4) Individual Chisq deviation
# ------------------------------
ax4 = fig.add_subplot(2, 2, 4)
ax4.scatter(range(nrecords), chisq, alpha=0.5, marker='.')
ax4.scatter(GSFCrejected, chisq[GSFCrejected], alpha=0.5, marker='.', label='GSFC invalid')
ax4.scatter(chi2rejected, chisq[chi2rejected], alpha=0.5, marker='x', label='Chi2 invalid')
ax4.legend(fontsize="x-small")
ax4.axhline(chth, color="gray", ls='-')
ax4.axhline(1e4, color="gray", ls='-')
annotes = range(nrecords)
af =  AnnoteFinder(range(nrecords), chisq, annotes, ax=ax4)
fig.canvas.mpl_connect('button_press_event', af)
ax4.set_yscale('log')
ax4.set_ylabel("Chisq of deviation")
ax4.set_xlabel("Record number")

fig.tight_layout()

invalid_records = GSFCrejected + chi2rejected
PH_ID_rejected = PHIDdata_KasPH[invalid_records]
PH_ID_invalid_total = np.append(PH_ID_noKas, PH_ID_rejected)
print("Final list of invalid PH_ID:",PH_ID_invalid_total)
print("Number of final Kas pulses", len(PH_ID_Kas)-len(PH_ID_rejected))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Final list of invalid PH_ID: [   10    14    22 ... 46734 46747 47017]
Number of final Kas pulses 9935


In [62]:
# exclude non-truly Kas pulses (PH-rejected, GSFC-rejected, chi2-rejected) from PKH-Kas file
remove_invalid_records(infile=fileph_KasPKH, ext=1, id_list=PH_ID_invalid_total, colname="PH_ID", outfile=fileph_Kas)

Selecting valid records
fselect infile=channel_1/pulse_chan1_KasPKH.fits+1 outfile=channel_1/pulse_chan1_Kas.fits clobber=yes expr='PH_ID!= 10'
Selecting valid records
iph=21/8772
fselect infile=channel_1/pulse_chan1_Kas.fits+1 outfile=pp1593612868.fits clobber=yes expr='PH_ID!= 14 && PH_ID!= 22 && PH_ID!= 25 && PH_ID!= 44 && PH_ID!= 46 && PH_ID!= 54 && PH_ID!= 55 && PH_ID!= 59 && PH_ID!= 61 && PH_ID!= 63 && PH_ID!= 71 && PH_ID!= 76 && PH_ID!= 79 && PH_ID!= 85 && PH_ID!= 87 && PH_ID!= 99 && PH_ID!= 111 && PH_ID!= 112 && PH_ID!= 116 && PH_ID!= 122'
Selecting valid records
iph=41/8772
fselect infile=channel_1/pulse_chan1_Kas.fits+1 outfile=pp1593612868.fits clobber=yes expr='PH_ID!= 125 && PH_ID!= 137 && PH_ID!= 139 && PH_ID!= 143 && PH_ID!= 152 && PH_ID!= 159 && PH_ID!= 162 && PH_ID!= 164 && PH_ID!= 173 && PH_ID!= 179 && PH_ID!= 184 && PH_ID!= 187 && PH_ID!= 194 && PH_ID!= 203 && PH_ID!= 204 && PH_ID!= 207 && PH_ID!= 218 && PH_ID!= 227 && PH_ID!= 234 && PH_ID!= 237'
Selecting valid reco

## 2.8) Create library of clean Kas

In [65]:
# build library Kas
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
comm = ("tesreconstruction Recordfile=" + fileph_Kas + " TesEventFile=" + tmpFile + " PulseLength=" + str(plen) + 
        " LibraryFile=" + libKas + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) + 
        " opmode=0 FilterMethod=" + F0orB0 + " clobber=yes EnergyMethod=" + method + 
        " OFLength=" + str(liblen) + " NoiseFile=" + noisefile +
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(MnKas_cmass) + " preBuffer=" + str(preBuffer))
run_comm(comm, "Building final library (Kas)")
os.remove(tmpFile)
print("##########################################")
print("Finished creation of Library of Kas events")
print("##########################################")

Building final library (Kas)
tesreconstruction Recordfile=channel_1/pulse_chan1_Kas.fits TesEventFile=channel_1/pp1593617817.fits PulseLength=4096 LibraryFile=channel_1/library_Kas_4096.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=0 FilterMethod=F0 clobber=yes EnergyMethod=OPTFILT OFLength=4096 NoiseFile=noise/noise_chan1_spec.fits XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml monoenergy=5894.40 preBuffer=0
##########################################
Finished creation of Library of Kas events
##########################################


### 2.7) Select PH_ID of (non)Ka1 pulses to later exclude them

In [ ]:
%%script false --no-raise-error

# Get the range where prob(Ka1)/prob(Ka2)>ratio
nbinsKas = 80
(PHminKa1,PHmaxKa1) = fit2GaussAndRatio(data=data_gauss.SIGNAL, a1=60, a2=100, mean1=5.85, mean2=5.86, 
                                       sig1=0.005, sig2=0.005, nbins1=nbinsKas, ratio=ratio, xlab="Reconstructed PH (a.u.)",
                                        xlim=(5.81,5.88), ylim=(0,130))
print("PHminKa1=",PHminKa1, "PHmaxKa1=", PHmaxKa1)

In [ ]:
%%script false --no-raise-error

#select PH_IDs for Ka1 (in [PHmin,PHmax] interval)
dataKa1 = dataKas_HR[(dataKas_HR.SIGNAL >= PHminKa1) & (dataKas_HR.SIGNAL <= PHmaxKa1)]
print("Kas pulses which are Ka1 pulses in PHminKa1,PHmaxKa1 interval=", len(dataKa1))

PH_ID_Ka1 = np.sort(dataKa1.PH_ID.to_list())
print("Example of Ka1 pulses:", PH_ID_Ka1[:20])
#PH_ID_Ka1_single = set(PH_ID_Ka1).intersection(dataKas_single_PH_ID) # good Ka1 pulses

#select PH_IDs to exclude non-Ka1 in single-pulse records
PH_ID_noKa1 = np.sort(list(set(dataKas_HR.PH_ID.to_list()).difference(PH_ID_Ka1)))
print("Kas pulses which are not Ka1 pulses:", len(PH_ID_noKa1))
print("Example of Kas pulses which are not Ka1 pulses:", PH_ID_noKa1[:20])

### 2.x) Extract Ka1 pulses to build a new more monochromatic library

In [ ]:
%%script false --no-raise-error

remove_invalid_records(infile=fileph_Kas, ext=1, id_list=PH_ID_noKa1, colname="PH_ID", outfile=fileph_Ka1)

### 2.x) Build the new library of Ka1 pulses

In [ ]:
# plot histogram of ADC max values of Kas and Ka1 files

#print("ADCmax,axis0=", np.amax(ADCdata_Kas, axis=0))
#print("ADCmax,axis1=", np.amax(ADCdata_Kas, axis=1))
#print(ADCdata_Kas)

f = fits.open(fileph_Ka1)
ADCdata_Ka1 = f["TESRECORDS"].data['ADC']
PHIDdata_Ka1 = f["TESRECORDS"].data['PH_ID']
ADCmax_Ka1 = np.amax(ADCdata_Ka1, axis=1)
f.close()
print("ADCdata_Ka1[0]=", ADCdata_Ka1[0])
print("ADCdata_Ka1[1]=", ADCdata_Ka1[1])

fig = plt.figure(figsize=(9,4))
ax1 = fig.add_subplot(1, 2, 1)
bin_heights, bin_borders, _ = ax1.hist(ADCmax_Kas, bins=5, alpha=0.4)
ax1.set_xlabel("Maximum value of Kas ADC in record")
ax1.set_ylabel("Number of records")
ax1.set_title("Histogram of max(ADC) in Kas")
PHmin = 22500 # ADC units to limit Kas lines
ax1.axvline(PHmin, linestyle="--", color="gray")
PHmax = 25000 # ADC units to limit Kas lines
ax1.axvline(PHmax, linestyle="--", color="gray")

ax2 = fig.add_subplot(1, 2, 2)
bin_heights, bin_borders, _ = ax2.hist(ADCmax_Ka1, bins=5, alpha=0.4)
ax2.set_xlabel("Maximum value of Ka1 ADC in record")
ax2.set_ylabel("Number of records")
ax2.set_title("Histogram of max(ADC) in Ka1")
PHmin = 22500 # ADC units to limit Kas lines
ax2.axvline(PHmin, linestyle="--", color="gray")
PHmax = 25000 # ADC units to limit Kas lines
ax2.axvline(PHmax, linestyle="--", color="gray")



In [ ]:
# Get list of REJECTED pulses  ---------- SELECT INVALID PULSES ---------------------
invalid_records = list()
nrecords = ADCdata_Ka1.shape[0]
lenrec = ADCdata_Ka1.shape[1]
len_ave = 5000
ave_Ka1 = np.zeros(len_ave)
fig = plt.figure(figsize=(6,3))
ax1 = fig.add_subplot(1, 1, 1)

#istart, iend, thresh = autoDeterminePulseWindowAndThreshold(ADCdata_Ka1,nbase=700,nsigma=100, numSamples=4200, plot=True)
istart, iend, thresh, ave_Ka1 = autoDeterminePulseWindowAndThreshold(ADCdata_Ka1, nbase=700, nsigma=100, numSamples=len_ave, 
                                                                 plot=True, ax=ax1)
icross = 999 + (thresh-ave_Ka1[999])/(ave_Ka1[1000]-ave_Ka1[999])

nobsln_Ka1 = np.zeros((ADCdata_Ka1.shape))
ax1.plot(icross,thresh,marker='x')

print("Pulse window in average:", istart, iend, thresh)

rejected = list()
rejected_comm = list()
for irec in range(nrecords):
    #rec0 = ADCdata_Ka1[irec,:] - np.mean(ADCdata_Ka1[irec,0:1000])
    base = np.mean(ADCdata_Ka1[irec,1000:1800])
    nobsln_Ka1[irec,:] = ADCdata_Ka1[irec,:] - base 
    #print("For irec=",irec,"base=",np.mean(nobsln_Ka1[irec,1000:1800]))
    rec_class = categorize(nobsln_Ka1[irec,:], istart, iend, thresh, joff=10) # initial offset is needed to avoid jitter effects
    #print("irec=", irec, "rec_class=", rec_class)
    if rec_class["rejected"] == 1:
        rejected.append(irec)
        rejected_comm.append(rec_class["rejected_comm"])
#ax1.plot(range(0,4200), nobsln_Ka1[145,1053:5253])
print("List of rejected pulses:", rejected)                        
print("Cause of rejection:", rejected_comm)            

invalid_records = rejected

In [ ]:


# align pulses with average record and calculate chisq
chisq = np.zeros(nrecords)
nobsln_Ka1_cut = np.zeros((nrecords, len_ave))
meanstd = 8.7
chth = 6.9e4 # threshld in Chisq to remove records
chth = 1e4
invalid_records2 = list()
icross_cut = list()
for irec in range(nrecords):
    s0 = np.where(nobsln_Ka1[irec,:] > thresh)[0][0]
    stSample = s0-istart
    fnSample = stSample + len_ave
    if stSample < 0:
        print("Possible pulse in preBuffer:")
        print("      For irec=",irec,"s0,ini,fin=", s0,stSample,fnSample)
        invalid_records2.append(irec)
        chisq[irec] = 1e6
        icross_cut.append(1000)
    else:
        nobsln_Ka1_cut[irec,:] = nobsln_Ka1[irec,stSample:fnSample]
    #    chisq[irec] = (chi2(nobsln_Ka1_cut[irec,stSample:(s0-10)], ave_Ka1[stSample:(s0-10)], meanstd) + 
    #                chi2(nobsln_Ka1_cut[irec,(s0+2):], ave_Ka1[(s0+2):], meanstd))
        ic = 999 + (thresh-nobsln_Ka1_cut[irec,999])/(nobsln_Ka1_cut[irec,1000]-nobsln_Ka1_cut[irec,999])
        icross_cut.append(ic)
        off = icross-ic
        xx = np.array(range(len_ave))
        off_func = interp1d(xx, nobsln_Ka1_cut[irec,:], kind='linear', fill_value='extrapolate')
        off_cut = off_func(xx-off)
        #chisq1 = chi2(nobsln_Ka1_cut[irec,0:(istart-10)], ave_Ka1[0:(istart-10)], meanstd)
        #chisq2 = chi2(nobsln_Ka1_cut[irec,(istart+2):], ave_Ka1[(istart+2):], meanstd)
        #chisq[irec] = chisq1 + chisq2
        chisq[irec] = chi2(off_cut, ave_Ka1, meanstd)
        if chisq[irec] > chth:
            invalid_records2.append(irec)

In [ ]:
# Plots of selection of pulses
plt.close()
# 1) Plot difference between average pulse and rejected/invalid/tocheck pulses
#test_pulses = rejected
test_pulses = [145, 214, 2531]
#test_pulses = [3484, 4130, 4206, 4425, 4645,4696]
test_pulses = [1592]

fig = plt.figure(figsize=(9,6))
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(range(len(ave_Ka1)), len(ave_Ka1)*[0], ls='')
for irej in test_pulses:
    index = test_pulses.index(irej)
    color = "C"+ str(index)
    #rej_record = nobsln_Ka1_cut[irej,:]
    xx = np.array(range(len_ave))
    off_func = interp1d(xx, nobsln_Ka1_cut[irej,:], kind='linear', fill_value='extrapolate')
    off_rej = off_func(xx-off)
    #diff_record = rej_record - ave_Ka1
    diff_record = off_rej - ave_Ka1
    ax1.plot(range(len_ave), diff_record, color=color, label=("Test pulse (" + str(irej) + ")"))
    #ax1.plot(range(0,istart-5), diff_record[0:istart-5], color=color)
    #ax1.plot(range(istart+5,len(ave_Ka1)), diff_record[istart+5:], label=("Test pulse (" + str(irej) + ")"), color=color)
ax1.axvline(istart, color="gray", ls="--")
ax1.axvline(iend, color="gray", ls="--")
#ax1.plot(range(len(ave_Ka1)), ave_Ka1, label="Average pulse")
#ax1.plot(range(len(ave_Ka1)), rej_record, label="Test Pulse")
ax1.set_title("Difference with Average pulse")
ax1.set_ylabel("Record in ADC - Average (a.u.)")
ax1.set_xlabel("Record sample")
ax1.legend(fontsize="xx-small", loc="right")
# 2) Average and test pulse
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(range(len(ave_Ka1)), ave_Ka1, label=("Average pulse"), color="black")
for irej in test_pulses:
    index = test_pulses.index(irej)
    color = "C"+ str(index)
    rej_record = nobsln_Ka1_cut[irej,:]
    ax2.plot(range(len(ave_Ka1)), rej_record, label=("Test pulse (" + str(irej) + ")"), color=color)
    ax2.plot(icross_cut[irej], thresh, color=color, marker="x")
    off = icross-icross_cut[irej]
    ax2.plot(np.arange(len(ave_Ka1))+off, rej_record, label=("Test pulse (" + str(irej) + ") offset"), color="C1", ls="--")
ax2.axvline(istart, color="gray", ls="--")
ax2.axvline(iend, color="gray", ls="--")
ax2.axhline(thresh, color="gray", ls="--")
ax2.set_title("Average & Test pulse")
ax2.set_ylabel("ADC (a.u.)")
ax2.set_xlabel("Record sample")
ax2.legend(fontsize="xx-small")
# 3) Cumulative Chisq deviation between average record and every record
ax3 = fig.add_subplot(2, 2, 3)
nofrecs = nrecs(chisq, chisq)
ax3.plot(nofrecs, chisq, marker='.', ls='')
ax3.fill_between(nofrecs, min(chisq), max(chisq), where=chisq > chth, color='gray', alpha=0.05)
ax3.set_yscale('log')
ax3.set_ylabel("Deviation")
ax3.set_xlabel("# records to remove (Dev>deviation)")

# 4) Individual Chisq deviation
ax4 = fig.add_subplot(2, 2, 4)
ax4.scatter(range(nrecords), chisq, alpha=0.5, marker='.')
ax4.scatter(invalid_records, chisq[invalid_records], alpha=0.5, marker='.', label='GSFC invalid')
ax4.scatter(invalid_records2, chisq[invalid_records2], alpha=0.5, marker='x', label='Chi2 invalid')
ax4.legend(fontsize="x-small")
ax4.axhline(chth, color="gray", ls='-')
annotes = range(nrecords)
af =  AnnoteFinder(range(nrecords), chisq, annotes, ax=ax4)
fig.canvas.mpl_connect('button_press_event', af)
ax4.set_yscale('log')
ax4.set_ylabel("Chisq of deviation")
ax4.set_xlabel("Record number")

fig.tight_layout()

print("Final list of invalid records:",invalid_records)

In [ ]:
# exclude invalid records from list of Ka1 pulses before creating the template
invalid_PHIDs = PHIDdata_Ka1[invalid_records]
remove_invalid_records(infile=fileph_Ka1, ext=1, id_list=invalid_PHIDs, colname="PH_ID", outfile=fileph_Ka1_clean)



In [ ]:
#%%script false --no-raise-error
tmpFile = resDir + "/" + "pp" + str(int(datetime.timestamp(datetime.now()))) + ".fits"
comm = ("tesreconstruction Recordfile=" + fileph_Ka1 + " TesEventFile=" + tmpFile + " PulseLength=" + str(liblen) +
        " LibraryFile=" + libKa1_cleanp + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) +
        " opmode=0" + " FilterMethod=" + F0orB0 + " clobber=yes" + " EnergyMethod=" + method +  " NoiseFile=" + noisefile + 
        " XMLFile=" + xmlfileSX + " monoenergy=" + str(MnKa1_cmass) + " preBuffer=" + str(preBuffer))
try:
    print("Building new library (Ka1)")
    print(comm)
    args = shlex.split(comm)
    check_call(args, stderr=STDOUT)
except:
    print("Error Building new library (Ka1) with command:\n", comm)
    os.remove(tmpFile)
    raise
os.remove(tmpFile)